In [1]:
from datasets.dnerf_3d_video import _load_data_from_json
from datasets.dnerf_3d_video import dynerf_isg_weight
from datasets.dnerf_3d_video import dynerf_ist_weight_nice
import os
import time
import torch

In [2]:
root_fp = '/home/loyot/workspace/SSD_1T/Datasets/NeRF/3d_vedio_datasets'
subject_id = 'flame_salmon_1'
split = 'train'
read_image = True
factor = 4
basedir = os.path.join(root_fp, subject_id)

In [3]:
(
    images,
    poses,
    timestamps,
    bounds,
    images_per_video,
    num_cameras,
    instrinc,
    render_poses,
    median_imgs,
) = _load_data_from_json(root_fp, subject_id, factor=factor, split=split, read_img=read_image)

poses shape:  (19, 3, 4)
loading video:


flame_salmon_1-cam20: 100%|██████████| 300/300 [00:02<00:00, 103.20it/s]


In [4]:
num_images = images.shape[0]
height = images.shape[1]
width = images.shape[2]
keyframes = False

In [5]:
print(height, width)

507 676


In [6]:
images.shape[0] * images.shape[1] * images.shape[2] 

1850752800

In [1]:
# Precompute ISG weights
t_s = time.time()
gamma = 1e-3 if keyframes else 2e-2
isg_weights = dynerf_isg_weight(
    images.view(-1, height, width, images.shape[-1]),
    median_imgs=median_imgs, gamma=gamma)
# Normalize into a probability distribution, to speed up sampling
isg_weights = (isg_weights.reshape(-1) / torch.sum(isg_weights))
torch.save(isg_weights, os.path.join(basedir, f"isg_weights.pt"))
t_e = time.time()
print(f"Computed {isg_weights.shape[0]} ISG weights in {t_e - t_s:.2f}s.")

NameError: name 'time' is not defined

In [7]:
# Precompute IST weights
t_s = time.time()
ist_weights = dynerf_ist_weight_nice(
    images.view(-1, height, width, images.shape[-1]),
    num_cameras=median_imgs.shape[0])
# Normalize into a probability distribution, to speed up sampling
ist_weights = (ist_weights.reshape(-1) / torch.sum(ist_weights))
torch.save(ist_weights, os.path.join(basedir, f"ist_weights.pt"))
t_e = time.time()
print(f"Computed {ist_weights.shape[0]} IST weights in {t_e - t_s:.2f}s.")

100%|██████████| 18/18 [05:08<00:00, 17.13s/it]


Computed 1850752800 IST weights in 315.41s.


In [8]:
ist_weights.shape

torch.Size([1850752800])

In [ ]:
if split == "train" and dset_type == 'llff':  # Only use importance sampling with DyNeRF videos
    if os.path.exists(os.path.join(datadir, f"isg_weights.pt")):
        self.isg_weights = torch.load(os.path.join(datadir, f"isg_weights.pt"))
        log.info(f"Reloaded {self.isg_weights.shape[0]} ISG weights from file.")
    else:
        # Precompute ISG weights
        t_s = time.time()
        gamma = 1e-3 if self.keyframes else 2e-2
        self.isg_weights = dynerf_isg_weight(
            imgs.view(-1, intrinsics.height, intrinsics.width, imgs.shape[-1]),
            median_imgs=self.median_imgs, gamma=gamma)
        # Normalize into a probability distribution, to speed up sampling
        self.isg_weights = (self.isg_weights.reshape(-1) / torch.sum(self.isg_weights))
        torch.save(self.isg_weights, os.path.join(datadir, f"isg_weights.pt"))
        t_e = time.time()
        log.info(f"Computed {self.isg_weights.shape[0]} ISG weights in {t_e - t_s:.2f}s.")

    if os.path.exists(os.path.join(datadir, f"ist_weights.pt")):
        self.ist_weights = torch.load(os.path.join(datadir, f"ist_weights.pt"))
        log.info(f"Reloaded {self.ist_weights.shape[0]} IST weights from file.")
    else:
        # Precompute IST weights
        t_s = time.time()
        self.ist_weights = dynerf_ist_weight(
            imgs.view(-1, self.img_h, self.img_w, imgs.shape[-1]),
            num_cameras=self.median_imgs.shape[0])
        # Normalize into a probability distribution, to speed up sampling
        self.ist_weights = (self.ist_weights.reshape(-1) / torch.sum(self.ist_weights))
        torch.save(self.ist_weights, os.path.join(datadir, f"ist_weights.pt"))
        t_e = time.time()
        log.info(f"Computed {self.ist_weights.shape[0]} IST weights in {t_e - t_s:.2f}s.")
